In [1]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [3]:
df = pd.read_csv("../data/molecular_biology_promoters.tsv",delimiter='\t')
X = df.iloc[:,:-1].to_numpy().astype("float")
y = df.iloc[:,-1].to_numpy().astype("int")

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
train_size = 0.7
dataset = 'molecular'

In [4]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot

# model

In [13]:
def fit_trees():
    learning_rate=0.1
    max_depth=3
    bins=8
    lam=100

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 10
    epoch= 100
    gp_epoch= 3
    verbose = 1
    tolerance=0.01

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch= 100
    alpha=0
    beta=1
    gammer=0

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

with open('./benchmark/'+dataset+'.csv','w') as f:
        f.writelines("")

for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    # train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
    # test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')

    # train_f1 = roc_auc_score(gtgp.y_one_hot.toarray(),(gtgp.train_p.T/np.sum(gtgp.train_p,axis=1)).T)
    # test_f1 = roc_auc_score(y_test_one_hot.toarray(),(gtgp.test_p.T/np.sum(gtgp.test_p,axis=1)).T)

    train_f1 = roc_auc_score(gtgp.y_one_hot.toarray(),gtgp.train_p)
    test_f1 = roc_auc_score(y_test_one_hot.toarray(),gtgp.test_p)


    with open('./benchmark/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

18 70 1
28 92 2
33 109 3
38 124 4
43 143 5
48 156 6
52 172 7
56 188 8
59 201 9
62 212 10
65 223 11
68 230 12
71 239 13
72 244 14
73 247 15
76 260 16
77 261 17
79 265 18
85 281 19
86 284 20
91 303 21
93 305 22
94 310 23
95 315 24
98 322 25
98 318 26
98 316 27
98 314 28
102 330 29
103 335 30
106 344 31
110 358 32
111 361 33
114 370 34
115 371 35
118 382 36
119 385 37
119 383 38
119 381 39
120 382 40
120 382 41
120 382 42
123 395 43
124 398 44
124 396 45
126 400 46
127 403 47
127 403 48
129 409 49
129 409 50
129 409 51
130 414 52
130 414 53
131 425 54
131 425 55
131 425 56
131 425 57
132 428 58
134 436 59
136 444 60
137 447 61
139 455 62
140 460 63
139 453 64
139 453 65
140 456 66
140 456 67
141 459 68
141 455 69
141 453 70
141 453 71
141 453 72
141 451 73
141 451 74
142 450 75
142 450 76
143 453 77
143 453 78
143 453 79
143 453 80
143 453 81
143 453 82
143 453 83
144 454 84
145 457 85
147 459 86
149 465 87
149 465 88
150 470 89
150 470 90
152 478 91
152 478 92
152 476 93
154 482 94
154 4

# Decision Tree

In [10]:
from sklearn.tree import DecisionTreeClassifier

In [11]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)

    num_trees = 1
    depth = clf.tree_.max_depth
    num_nodes = clf.tree_.node_count

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    train_roc = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
    test_roc = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))

    with open('./benchmark_DC/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [12]:
import xgboost

In [38]:
with open('./benchmark_xgb/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # xgb = xgboost.XGBClassifier(n_estimators=100,max_depth=6)
        # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
        xgb = xgboost.XGBClassifier(n_estimators=1000)
        xgb.fit(X_train,y_train)

        import json

        def item_generator(json_input, lookup_key):
            if isinstance(json_input, dict):
                for k, v in json_input.items():
                    if k == lookup_key:
                        yield v
                    else:
                        yield from item_generator(v, lookup_key)
            elif isinstance(json_input, list):
                for item in json_input:
                    yield from item_generator(item, lookup_key)

        def tree_depth(json_text):
            json_input = json.loads(json_text)
            depths = list(item_generator(json_input, 'depth'))
            return max(depths) + 1 if len(depths) != 0 else 1

        train_acc = accuracy_score(y_train,xgb.predict(X_train))
        test_acc = accuracy_score(y_test,xgb.predict(X_test))

        # train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),xgb.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),xgb.predict_proba(X_test))

        booster = xgb.get_booster()

        tree_df = booster.trees_to_dataframe()
        depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
        num_trees = len(depths)
        depth = np.average(depths)
        num_nodes = len(tree_df)

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

In [27]:
with open('./benchmark_GBDT/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # clf = GradientBoostingClassifier(n_estimators=1000)
        clf = GradientBoostingClassifier()
        clf.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        # train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))


        num_trees = len([ est for ests in clf.estimators_ for est in ests])
        depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
        num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
with open('./benchmark_RF/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # rfc = RandomForestClassifier(n_estimators=100)
        rfc = RandomForestClassifier()
        rfc.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,rfc.predict(X_train))
        test_acc = accuracy_score(y_test,rfc.predict(X_test))
        # train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')
        
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),rfc.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),rfc.predict_proba(X_test))

        num_trees = len(rfc.estimators_)
        depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
        num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)